In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder, MinMaxScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.metrics import classification_report, precision_recall_curve, auc
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import warnings
warnings.filterwarnings("ignore")

In [2]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

print(device)

cuda


In [3]:
data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/VAE-CTAB-GAN/Real_Datasets/train_category_1.csv"
df = pd.read_csv(data_path)

categorical_cols = df.select_dtypes(include='object').columns.tolist()
df_encoded = pd.get_dummies(df, columns=categorical_cols)

continuous_cols = df_encoded.select_dtypes(include=['float64', 'int64']).columns.difference(['loan_status'])
scaler = MinMaxScaler(feature_range=(-1,1))
df_encoded[continuous_cols] = scaler.fit_transform(df_encoded[continuous_cols])

X = df_encoded.drop(columns=["loan_status"]).values.astype(np.float32)
y = df_encoded["loan_status"].values

In [4]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim=64):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(64, latent_dim)
        self.fc_logvar = nn.Linear(64, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Tanh() 
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = VAE(input_dim=X.shape[1],latent_dim=64).to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

X_tensor = torch.tensor(X, dtype=torch.float32)
train_loader = DataLoader(TensorDataset(X_tensor), batch_size=64, shuffle=True)

def vae_loss(recon_x, x, mu, logvar, beta):
    recon = loss_fn(recon_x, x)
    kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon + beta*kl

In [6]:
for epoch in tqdm(range(500),desc="Epoch :"):
    vae.train()
    total_loss = 0
    beta = min(1.0, epoch / 100)
    
    for batch in train_loader:
        x = batch[0].to(device)
        optimizer.zero_grad()
        recon, mu, logvar = vae(x)
        loss = vae_loss(recon, x, mu, logvar,beta)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch ::   0%|          | 1/500 [00:07<1:02:47,  7.55s/it]

Epoch 1, Loss: 35.6052


Epoch ::   0%|          | 2/500 [00:19<1:24:23, 10.17s/it]

Epoch 2, Loss: 34.5979


Epoch ::   1%|          | 3/500 [00:26<1:13:45,  8.90s/it]

...

Epoch 498, Loss: 208.0425


Epoch :: 100%|█████████▉| 499/500 [1:08:37<00:08,  8.37s/it]

Epoch 499, Loss: 208.0610


Epoch :: 100%|██████████| 500/500 [1:08:45<00:00,  8.25s/it]

Epoch 500, Loss: 208.0440


In [7]:
def inverse_one_hot(samples_df, original_df, categorical_cols):
    decoded_df = samples_df.copy()

    for col in categorical_cols:
        # 원래 one-hot으로 생성된 컬럼 리스트 추출
        one_hot_cols = [c for c in samples_df.columns if c.startswith(col + '_')]
        if not one_hot_cols:
            continue
        
        # one-hot 복원: 가장 큰 값의 인덱스를 카테고리로
        col_values = pd.Series(np.argmax(samples_df[one_hot_cols].values, axis=1))
        col_categories = [c.split('_', 1)[1] for c in one_hot_cols]
        decoded_df[col] = col_values.map(lambda x: col_categories[x])

        # one-hot 컬럼 제거
        decoded_df = decoded_df.drop(columns=one_hot_cols)

    return decoded_df


In [8]:
vae.eval()
with torch.no_grad():
    z = torch.randn(540000, 32).to(device)
    samples = vae.decoder(z).cpu().numpy()

# 🔄 Inverse Transform
columns = df_encoded.drop(columns=["loan_status"]).columns
samples_df = pd.DataFrame(samples, columns=columns)
samples_df["loan_status"] = 1

samples_df[continuous_cols] = scaler.inverse_transform(samples_df[continuous_cols])
samples_decoded = inverse_one_hot(samples_df, df, categorical_cols)

samples_decoded.head()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (540000x32 and 64x64)

In [ ]:
for col in samples_decoded.columns:
    print(f"col = {col} :\n {samples_decoded[col].unique()}")

col = last_fico_range_high :
 [569.7965]
col = annual_inc :
 [92879.29]
col = dti :
 [21.072958]
col = mo_sin_old_rev_tl_op :
 [170.18124]
col = revol_util :
 [0.53745276]
col = int_rate :
 [0.15711126]
col = installment :
 [471.18198]
col = avg_cur_bal :
 [11042.185]
col = revol_bal :
 [16306.236]
col = total_pymnt :
 [8592.611]
col = total_pymnt_inv :
 [8580.304]
col = funded_amnt :
 [15890.482]
col = loan_amnt :
 [15894.932]
col = credit_history_years :
 [15.6867895]
col = term_months :
 [45.496716]
col = loan_status :
 [1]
col = debt_settlement_flag :
 ['settlement_flag_N']
col = sub_grade :
 ['grade_C4']
col = home_ownership :
 ['ownership_RENT']
col = purpose :
 ['debt_consolidation']
col = grade :
 ['C']


In [ ]:
save_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/VAE-CTAB-GAN/Fake_Datasets/"
samples_decoded.to_csv(path_or_buf=save_path+"VAE.csv", index=False)

In [ ]:
sample_data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/VAE-CTAB-GAN/Fake_Datasets/VAE.csv"

fake = pd.read_csv(sample_data_path)

In [ ]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540000 entries, 0 to 539999
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   last_fico_range_high  540000 non-null  float64
 1   annual_inc            540000 non-null  float64
 2   dti                   540000 non-null  float64
 3   mo_sin_old_rev_tl_op  540000 non-null  float64
 4   revol_util            540000 non-null  float64
 5   int_rate              540000 non-null  float64
 6   installment           540000 non-null  float64
 7   avg_cur_bal           540000 non-null  float64
 8   revol_bal             540000 non-null  float64
 9   total_pymnt           540000 non-null  float64
 10  total_pymnt_inv       540000 non-null  float64
 11  funded_amnt           540000 non-null  float64
 12  loan_amnt             540000 non-null  float64
 13  credit_history_years  540000 non-null  float64
 14  term_months           540000 non-null  float64
 15  